# Setup

In [ ]:
import os
import pandas as pd

from rrg._data import (
    DEFAULT_DICOM_ID_COL,
    DEFAULT_FINDINGS_COL,
    DEFAULT_IMG_PROJ_KEY,
    DEFAULT_IMPRESSION_COL,
    DEFAULT_LABELS,
    DEFAULT_PATIENT_ID_COL,
    DEFAULT_SPLIT_COL,
    DEFAULT_STUDY_ID_COL,
    DEFAULT_VIEW_COL,
    DEFAULT_VIEW_ORDER,
    get_per_study_data,
    get_split_features,
    get_split_samples,
)


# helper functions
def extract_image_path(row):
    prefix = "/opt/gpudata/mimic-cxr/files/"
    subject_id = str(row["subject_id"])
    study_id = str(row["study_id"])
    dicom_id = str(row["dicom_id"])
    # path = "p" + str(row["subject_id"])[:2] + "/p" + str(row["subject_id"]) + "/s" + str(row["study_id"]) + "/" + str(row["dicom_id"]) +".jpg"
    path = f"p{subject_id[:2]}/p{subject_id}/s{study_id}/{dicom_id}.jpg"
    return prefix + path

# Create Dataset

In [ ]:
# csv files 
split_csv = "/opt/gpudata/mimic-cxr/mimic-cxr-2.0.0-split.csv"
metadata_csv = "/opt/gpudata/mimic-cxr/mimic-cxr-2.0.0-metadata.csv"
true_label_csv = "/opt/gpudata/mimic-cxr/mimic-cxr-2.0.0-chexpert.csv"
predicted_label_csv = "/opt/gpudata/rrg-data-2/image-labels/pred_pr.csv"
report_csv = "/opt/gpudata/mimic-cxr/mimic_cxr_sectioned.csv"

patient_id_col = DEFAULT_PATIENT_ID_COL
study_id_col = DEFAULT_STUDY_ID_COL
dicom_id_col = DEFAULT_DICOM_ID_COL
split_col = DEFAULT_SPLIT_COL
view_col = DEFAULT_VIEW_COL
findings_col = DEFAULT_FINDINGS_COL
impression_col = DEFAULT_IMPRESSION_COL
labels = DEFAULT_LABELS
view_order = DEFAULT_VIEW_ORDER
split_remap = {
        "train": "retrieval",
        "validate": "retrieval",
        "test": "inference",
    }

DEFAULT_OTHER_COL = "Other"

In [ ]:
mimic_cxr_section = pd.read_csv(report_csv) 

## findings

In [ ]:
# Filter dataset to only those with given section type
section_type = "findings"
if section_type == "findings":
    report_cols = [findings_col]
elif section_type == "impression":
    report_cols = [impression_col]
elif section_type in ["both", "findings-intersect", "impression-intersect"]:
    report_cols = [findings_col, impression_col]
else:
    raise ValueError(f"Unknown section type: {section_type}")

retrieval_df = get_per_study_data(
        split_csv=split_csv,
        metadata_csv=metadata_csv,
        label_csv=true_label_csv,
        report_csv=report_csv,
        patient_id_col=patient_id_col,
        study_id_col=study_id_col,
        dicom_id_col=dicom_id_col,
        split_col=split_col,
        view_col=view_col,
        labels=labels,
        view_order=view_order,
        report_cols=[findings_col, impression_col],
        split_remap=split_remap,
    )

inference_df = get_per_study_data(
        split_csv=split_csv,
        metadata_csv=metadata_csv,
        label_csv=predicted_label_csv,
        report_csv=report_csv,
        patient_id_col=patient_id_col,
        study_id_col=study_id_col,
        dicom_id_col=dicom_id_col,
        split_col=split_col,
        view_col=view_col,
        labels=labels,
        view_order=view_order,
        report_cols=report_cols,
        split_remap=split_remap,
    )

In [ ]:
mask = retrieval_df[report_cols].notna().all(axis=1)
retrieval_df = retrieval_df[mask].reset_index(drop=True).copy()
inference_df = inference_df[mask].reset_index(drop=True).copy()
add_other_label = True
other_col = DEFAULT_OTHER_COL

inference_samples = get_split_samples(
        sample_df=inference_df,
        split_col=split_col,
    )["inference"]

In [ ]:
inference_samples["dicom_path"] = inference_samples.apply(extract_image_path, axis=1)

In [ ]:
inference_samples["study_id"].nunique()

In [ ]:
inference_samples[inference_samples["findings"].isnull()]

In [ ]:
# replace savepath if needed
find_savepath = "/opt/gpudata/rrg-data-2/inference-all/inference_findings_data.csv"
inference_samples[["dicom_path", "study_id", "dicom_id", "findings"]].to_csv(find_savepath)

## impression

In [ ]:
# Filter dataset to only those with given section type
section_type = "impression"
if section_type == "findings":
    report_cols = [findings_col]
elif section_type == "impression":
    report_cols = [impression_col]
elif section_type in ["both", "findings-intersect", "impression-intersect"]:
    report_cols = [findings_col, impression_col]
else:
    raise ValueError(f"Unknown section type: {section_type}")

retrieval_df = get_per_study_data(
        split_csv=split_csv,
        metadata_csv=metadata_csv,
        label_csv=true_label_csv,
        report_csv=report_csv,
        patient_id_col=patient_id_col,
        study_id_col=study_id_col,
        dicom_id_col=dicom_id_col,
        split_col=split_col,
        view_col=view_col,
        labels=labels,
        view_order=view_order,
        report_cols=[findings_col, impression_col],
        split_remap=split_remap,
    )

inference_df = get_per_study_data(
        split_csv=split_csv,
        metadata_csv=metadata_csv,
        label_csv=predicted_label_csv,
        report_csv=report_csv,
        patient_id_col=patient_id_col,
        study_id_col=study_id_col,
        dicom_id_col=dicom_id_col,
        split_col=split_col,
        view_col=view_col,
        labels=labels,
        view_order=view_order,
        report_cols=report_cols,
        split_remap=split_remap,
    )

In [ ]:
mask = retrieval_df[report_cols].notna().all(axis=1)
retrieval_df = retrieval_df[mask].reset_index(drop=True).copy()
inference_df = inference_df[mask].reset_index(drop=True).copy()
add_other_label = True

inference_samples = get_split_samples(
        sample_df=inference_df,
        split_col=split_col,
    )["inference"]

In [ ]:
inference_samples["dicom_path"] = inference_samples.apply(extract_image_path, axis=1)

In [ ]:
inference_samples["study_id"].nunique()

In [ ]:
inference_samples[inference_samples["impression"].isnull()]

In [ ]:
# replace savepath if needed
inf_savepath = "/opt/gpudata/rrg-data-2/inference-all/inference_impression_data.csv"
inference_samples[["dicom_path", "study_id", "dicom_id", "impression"]].to_csv(inf_savepath)

# Results

In [ ]:
findings_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inference_findings_data.csv")
impression_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inference_impression_data.csv")

print(findings_df.head())
print(impression_df.head())

find_sids = findings_df["study_id"].tolist()
imp_sids = impression_df["study_id"].tolist()

## rgrg

In [ ]:
rgrg_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/rgrg/generations_findings.csv")
assert set(rgrg_results.study_id.tolist()) == set(find_sids)

## cxrmate

In [ ]:
cxrmate_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-mate/generations.csv")
cxrmate_findings = cxrmate_results[cxrmate_results["study_id"].isin(find_sids)][["study_id", "findings"]]

assert set(cxrmate_findings["study_id"]) == set(find_sids)
assert any(cxrmate_findings["findings"].notnull())

cxrmate_impression = cxrmate_results[cxrmate_results["study_id"].isin(imp_sids)][["study_id", "impression"]]

assert set(cxrmate_impression.study_id) == set(imp_sids)
assert any(cxrmate_impression["impression"].notnull())

cxrmate_both = pd.DataFrame.merge(cxrmate_findings, cxrmate_impression, on="study_id")
assert set(cxrmate_both.study_id) == set(set(find_sids).intersection(set(imp_sids)))

# findings
cxrmate_findings_final = cxrmate_findings.merge(findings_df, on="study_id")
cxrmate_findings_final.rename(columns={"findings_x" : "generated_text", "findings_y" : "actual_text"}, inplace=True)
cxrmate_findings_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-mate/generations_findings.csv")

# impression
cxrmate_impression_final = cxrmate_impression.merge(impression_df, on="study_id")
cxrmate_impression_final.rename(columns={"impression_x" : "generated_text", "impression_y" : "actual_text"}, inplace=True)
cxrmate_impression_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-mate/generations_impression.csv")

# both
def create_actual_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"

def create_generated_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"

full_reports_actual = pd.merge(findings_df, impression_df, on="study_id")
full_reports_actual["actual_text"] = full_reports_actual.apply(create_actual_report, axis=1)

# findings and impression
cxrmate_both["generated_text"] = cxrmate_both.apply(create_generated_report, axis=1)
cxrmate_both_final = cxrmate_both.merge(full_reports_actual[["study_id", "actual_text"]], on="study_id")
cxrmate_both_final[["study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-mate/generations_full_reports.csv")

## chexagent

In [ ]:
chexagent_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/chexagent/generations.csv")
chexagent_findings = chexagent_results[chexagent_results["study_id"].isin(find_sids)][["study_id", "findings"]]
chexagent_impression = chexagent_results[chexagent_results["study_id"].isin(imp_sids)][["study_id", "impression"]]
chexagent_both = pd.DataFrame.merge(chexagent_findings, chexagent_impression, on="study_id")

# findings

chexagent_findings_final = chexagent_findings.merge(findings_df, on="study_id")
chexagent_findings_final.rename(columns={"findings_x" : "generated_text", "findings_y" : "actual_text"}, inplace=True)
chexagent_findings_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/chexagent/generations_findings.csv")

# impression

chexagent_impression_final = chexagent_impression.merge(impression_df, on="study_id")
chexagent_impression_final.rename(columns={"impression_x" : "generated_text", "impression_y" : "actual_text"}, inplace=True)
chexagent_impression_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/chexagent/generations_impression.csv")

# both

def create_actual_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"

def create_generated_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"

full_reports_actual = pd.merge(findings_df, impression_df, on="study_id")
full_reports_actual["actual_text"] = full_reports_actual.apply(create_actual_report, axis=1)

# findings and impression
chexagent_both["generated_text"] = chexagent_both.apply(create_generated_report, axis=1)
chexagent_both_final = chexagent_both.merge(full_reports_actual[["study_id", "actual_text"]], on="study_id")
chexagent_both_final[["study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/chexagent/generations_full_reports.csv")

## cxr-repair

In [ ]:
cxr_repair_generations = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-repair/generations_full.csv")
# filter and add actual text from impression
cxr_repair_gen_final = cxr_repair_generations.merge(impression_df, on="study_id")
cxr_repair_gen_final = cxr_repair_gen_final.drop_duplicates("study_id").rename(columns={"impression" : "actual_text"})
cxr_repair_gen_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-repair/generations_impression.csv")

## cxr-redone

In [ ]:
cxr_redone_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-redone/generations.csv")
xrem_sids = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/x-rem/xrem_sids.csv")
cxr_redone_results = cxr_redone_results.merge(xrem_sids, left_index=True, right_index=True, how="inner")
cxr_redone_final = cxr_redone_results.merge(impression_df, on="study_id")
cxr_redone_final = cxr_redone_final.drop_duplicates("study_id").rename(
    columns={
        "impression" : "actual_text",
        "Report Impression": "generated_text"
    })
cxr_redone_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-redone/generations_impression.csv")

## xrem

In [ ]:
xrem_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/x-rem/final_results_filtered.csv")
xrem_sids = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/x-rem/xrem_sids.csv")
xrem_results_all = xrem_results.merge(xrem_sids, left_index=True, right_index=True, how="inner")
xrem_results_all = xrem_results_all.merge(impression_df, on="study_id")
xrem_results_all = xrem_results_all.drop_duplicates("study_id")
xrem_results_all[["dicom_id", "study_id", "impression", "Report Impression"]].rename(
    columns={
        "impression" : "actual_text",
        "Report Impression" : "generated_text"
    }).to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/x-rem/generations_impression.csv")

# Create inference results

In [ ]:
def extract_exp_info(experiment):
    tabs = experiment.split("_")
    return {
         "section" : "_".join(tabs[1:-1])
    }

# extract_exp_info("generations_full_reports_METRICS.csv")
# extract_exp_info("generations_findings_METRICS.csv")

# our results
def extract_results(df):
    results = {}
    results["run"] = ""
    results["BLEU4"] = df["bleu4"].mean() * 100.0
    results["RougeL"]= df["rougeL"].mean() * 100.0
    results["BertScore"] = df["bertscore"].mean() * 100.0
    results["f1-RadGraph"] = df["f1radgraph"].mean() * 100.0
    results["f1-CheXbert"] = df["f1chexbert"].mean() * 100.0
    return results

In [ ]:
results_dir = "/opt/gpudata/rrg-data-2/inference-all/inf-results/"
all_results = {
    "findings" : [],
    "impression" : [],
    "both" : [],
}

for foldername in os.listdir(results_dir):
    print(foldername)
    folderpath = os.path.join(results_dir, foldername)
    if os.path.isdir(folderpath):
        for filename in os.listdir(folderpath):
            if "METRICS" in filename:
                res = extract_exp_info(filename)
                filepath = os.path.join(results_dir, os.path.join(foldername, filename))
                df = pd.read_csv(filepath)

                # aggregate results
                results = {}
                run_name = f"{foldername}_{res['section']}"
                results["run"] = run_name
                results["BLEU4"] = df["bleu4"].mean() * 100.0
                results["RougeL"]= df["rougeL"].mean() * 100.0
                results["BertScore"] = df["bertscore"].mean() * 100.0
                results["f1-RadGraph"] = df["f1radgraph"].mean() * 100.0
                results["f1-CheXbert"] = df["f1chexbert"].mean() * 100.0

                if "findings" in filename:
                    all_results["findings"].append(results)
                elif "impression" in filename:
                    all_results["impression"].append(results)
                else:
                    all_results["both"].append(results)

### findings

In [ ]:
# findings
findings_best = pd.read_csv("/opt/gpudata/rrg-data-2/exp-findings/exp-model/Mistral-7B-Instruct-v0.3_exact_pred-label_simple_top-5_findings_METRICS.csv")
findings_result = extract_results(findings_best)
findings_result["run"] = "label_boosted_rag_findings"
pd.DataFrame(findings_result, index=[0])

In [ ]:
pd.DataFrame.from_dict(all_results["findings"])

### impression

In [ ]:
# impression
findings_best = pd.read_csv("/opt/gpudata/rrg-data-2/exp-impression/exp-model/Mistral-7B-Instruct-v0.3_exact_pred-label_simple_top-5_impression_METRICS.csv")
findings_result = extract_results(findings_best)
findings_result["run"] = "label_boosted_rag_impression"
pd.DataFrame(findings_result, index=[0])

In [ ]:
pd.DataFrame.from_dict(all_results["impression"])

### both

In [ ]:
pd.DataFrame.from_dict(all_results["both"])

In [ ]:
# create a list of image paths to push to rgrg for inference

In [ ]:
from rrg._data import (
    DEFAULT_DICOM_ID_COL,
    DEFAULT_FINDINGS_COL,
    DEFAULT_IMG_PROJ_KEY,
    DEFAULT_IMPRESSION_COL,
    DEFAULT_LABELS,
    DEFAULT_PATIENT_ID_COL,
    DEFAULT_SPLIT_COL,
    DEFAULT_STUDY_ID_COL,
    DEFAULT_VIEW_COL,
    DEFAULT_VIEW_ORDER,
    get_per_study_data,
    get_split_features,
    get_split_samples,
)

In [ ]:
split_csv = "/opt/gpudata/mimic-cxr/mimic-cxr-2.0.0-split.csv"
metadata_csv = "/opt/gpudata/mimic-cxr/mimic-cxr-2.0.0-metadata.csv"
true_label_csv = "/opt/gpudata/mimic-cxr/mimic-cxr-2.0.0-chexpert.csv"
predicted_label_csv = "/opt/gpudata/rrg-data-2/image-labels/pred_pr.csv"
report_csv = "/opt/gpudata/mimic-cxr/mimic_cxr_sectioned.csv"
patient_id_col = DEFAULT_PATIENT_ID_COL
study_id_col = DEFAULT_STUDY_ID_COL
dicom_id_col = DEFAULT_DICOM_ID_COL
split_col = DEFAULT_SPLIT_COL
view_col = DEFAULT_VIEW_COL
findings_col = DEFAULT_FINDINGS_COL
impression_col = DEFAULT_IMPRESSION_COL
labels = DEFAULT_LABELS
view_order = DEFAULT_VIEW_ORDER
section_type = "both"
split_remap = {
        "train": "retrieval",
        "validate": "retrieval",
        "test": "inference",
    }

In [ ]:
reports_df = pd.read_csv(report_csv)
reports_df

In [ ]:
reports_df.iloc[4]["findings"]

In [ ]:
reports_df[reports_df["study_id"] == 53492798]

In [ ]:
if section_type == "findings":
    report_cols = [findings_col]
elif section_type == "impression":
    report_cols = [impression_col]
elif section_type == "both":
    report_cols = [findings_col, impression_col]
else:
    raise ValueError(f"Unknown section type: {section_type}")

# Load and merge data relative to true labels
retrieval_df = get_per_study_data(
    split_csv=split_csv,
    metadata_csv=metadata_csv,
    label_csv=true_label_csv,
    report_csv=report_csv,
    patient_id_col=patient_id_col,
    study_id_col=study_id_col,
    dicom_id_col=dicom_id_col,
    split_col=split_col,
    view_col=view_col,
    labels=labels,
    view_order=view_order,
    report_cols=report_cols,
    split_remap=split_remap,
)

In [ ]:
inference_df = get_per_study_data(
        split_csv=split_csv,
        metadata_csv=metadata_csv,
        label_csv=predicted_label_csv,
        report_csv=report_csv,
        patient_id_col=patient_id_col,
        study_id_col=study_id_col,
        dicom_id_col=dicom_id_col,
        split_col=split_col,
        view_col=view_col,
        labels=labels,
        view_order=view_order,
        report_cols=report_cols,
        split_remap=split_remap,
    )

In [ ]:
inf_only_df = inference_df[inference_df["split"] == "inference"]
inf_only_df.columns

In [ ]:
inf_only_df

In [ ]:
def extract_image_path(row):
    prefix = "/opt/gpudata/mimic-cxr/files/"
    path = "p" + str(row["subject_id"])[:2] + "/p" + str(row["subject_id"]) + "/s" + str(row["study_id"]) + "/" + str(row["dicom_id"]) +".jpg"
    return prefix + path
    
inf_only_df["image_path"] = inf_only_df.apply(extract_image_path, axis=1)

In [ ]:
inf_only_df.head()

In [ ]:
path = inf_only_df[inf_only_df["study_id"] == 53492798]["image_path"].tolist()
print(path[0])

In [ ]:
print(inf_only_df[inf_only_df["study_id"] == 53492798]["findings"].tolist()[0])

In [ ]:
print(inf_only_df[inf_only_df["study_id"] == 53492798]["impression"].tolist()[0])

In [ ]:
from PIL import Image
im = Image.open(path[0])
base_width = 512
wpercent = (base_width / float(im.size[0]))
hsize = int((float(im.size[1]) * float(wpercent)))
im.resize((base_width, hsize), Image.Resampling.LANCZOS)

In [ ]:
inf_find_only = inf_only_df[inf_only_df["findings"].notnull()]
print(f"Size of the findings only inference dataset: {len(inf_find_only)}")
# inf_find_only.head()
inf_find_final = inf_find_only.rename(columns={"findings" : "actual_findings"})
inf_find_final[["study_id", "image_path" ,"actual_findings"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inference_findings_data.csv")

In [ ]:
inf_imp_only = inf_only_df[inf_only_df["impression"].notnull()]
print(f"Size of the findings only inference dataset: {len(inf_imp_only)}")
# inf_imp_only.head()
inf_imp_final = inf_imp_only.rename(columns={"impression" : "actual_impression"})
inf_imp_final[["study_id", "image_path" ,"actual_impression"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inference_impression_data.csv")

## Results

In [ ]:
rgrg_results_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/rgrg/generations_METRICS.csv")

In [ ]:
rgrg_results_df

In [ ]:
# aggregate results
results = {
    
}
df = rgrg_results_df
results["BLEU4"] = df["bleu4"].mean() * 100.0
results["RougeL"]= df["rougeL"].mean() * 100.0
results["BertScore"]= df["bertscore"].mean() * 100.0
results["f1-RadGraph"]= df["f1radgraph"].mean() * 100.0
results["f1-CheXbert"]= df["f1chexbert"].mean() * 100.0

In [ ]:
print(results)

In [ ]:
#5.847507  22.483156  86.516446    21.026496    45.154811

In [ ]:
# Study ids used in our method BoostedRAG
inf_path = "/opt/gpudata/rrg-data-2/inference-all/inference_set_image_paths.csv"
inf_df = pd.read_csv(inf_path)
brag_inf_sids = inf_df["study_id"].tolist()

In [ ]:
inf_df

In [ ]:
# CXR-REPAIR data preprocessing
cxr_mimic_train = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/cxr-repair/mimic_train_full.csv")
cxr_mimic_test = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/cxr-repair/mimic_test_full.csv")

In [ ]:
# how many of our study ids belong to cxr train
cxr_mimic_train[cxr_mimic_train["study_id"].isin(brag_inf_sids)]

In [ ]:
# how many of our study ids are found in cxr test
cxr_mimic_test[cxr_mimic_test["study_id"].isin(brag_inf_sids)].drop_duplicates("study_id")

In [ ]:
# how many of our study ids are found in cxr test impressions 
cxr_mimic_test_imp = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/cxr-repair/mimic_test_impressions.csv")
cxr_mimic_test_imp[cxr_mimic_test_imp["study_id"].isin(brag_inf_sids)].drop_duplicates("study_id")

In [ ]:
# how many of our study ids are found in cxr test impressions after filtering
cxr_mimic_test_imp = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/cxr-repair/mimic_test_impressions.csv", index_col=[0])
filtered = cxr_mimic_test_imp[cxr_mimic_test_imp["report"].notnull()]
filt_sids = filtered[filtered["study_id"].isin(brag_inf_sids)]["study_id"].tolist()
# filtered[filtered["study_id"].isin(brag_inf_sids)]
filtered

In [ ]:
filtered_dupe = filtered.drop_duplicates("study_id")
filtered_dupe[filtered_dupe["study_id"].isin(brag_inf_sids)]

In [ ]:
filt_left_out = set(brag_inf_sids) - set(filt_sids)
cxr_mimic_test_imp[cxr_mimic_test_imp["study_id"].isin(filt_left_out)].sort_values("study_id", kind="stable")

In [ ]:
# cxr reports to generate on
reports_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/cxr-repair/bootstrapped_test_set/reports.csv")

In [ ]:
reports_df.drop_duplicates("study_id")

In [ ]:
reports_sids = reports_df["study_id"].tolist()
len(set(brag_inf_sids) - set(reports_sids))

In [ ]:
filtered

In [ ]:
h5_sids_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/cxr-repair/sids_h5.csv")

In [ ]:
h5_sids_df

In [ ]:
set(brag_inf_sids) - set(h5_sids_df.preds_sids)

In [ ]:
# full cxr-repair generations
full_gen_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-repair/generations_full.csv")

In [ ]:
full_gen_df

In [ ]:
full_gen_df.drop_duplicates("study_id")

In [ ]:
_gen_df = full_gen_df[full_gen_df['study_id'].isin(brag_inf_sids)].drop_duplicates("study_id")

In [ ]:
gen_df = _gen_df.merge(inf_df[["study_id", "actual_text"]], on="study_id")

In [ ]:
gen_df[["study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-repair/generations.csv")

In [ ]:
gen_df[["study_id", "actual_text", "generated_text"]]

In [ ]:
from rich import print
print(gen_df.iloc[0]["actual_text"])

## Impression

In [ ]:
impression_inf_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inference_impression_data.csv")

In [ ]:
impression_inf_df

In [ ]:
cxr_gen_full = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-repair/generations_full.csv")

In [ ]:
cxr_gen_full

## Results

In [ ]:
cxr_repair_results_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-repair/generations_METRICS.csv")

In [ ]:
cxr_repair_results_df

In [ ]:
# aggregate results
results = {
    
}
df = cxr_repair_results_df
results["BLEU4"] = df["bleu4"].mean() * 100.0
results["RougeL"]= df["rougeL"].mean() * 100.0
results["BertScore"]= df["bertscore"].mean() * 100.0
results["f1-RadGraph"]= df["f1radgraph"].mean() * 100.0
results["f1-CheXbert"]= df["f1chexbert"].mean() * 100.0

In [ ]:
print(results)

In [ ]:
import torch  
from torchvision.transforms import v2
import transformers
from datasets import Dataset, Features, Value, Image
from torch.utils.data import DataLoader

In [ ]:
image_paths = inf_only_df["image_path"].tolist()
study_ids = inf_only_df["study_id"].tolist()
print(f"Size of the inference set: {len(image_paths)}")

In [ ]:
dataset_dict = {
    "image_path" : image_paths,
    "image" : image_paths,
    "study_id" : study_ids
}

In [ ]:
features = Features(
    {
        "image_path" : Value("string"),
        "image" : Image(),
        "study_id": Value("string"),
    }
)

In [ ]:
hf_dataset = Dataset.from_dict(dataset_dict, features=features)

In [ ]:
from tqdm import trange
for i in trange(len(hf_dataset)):
    print(hf_dataset[i])
    break

In [ ]:
model_path = "/opt/gpudata/rrg-data-2/inference-all/inf-models/cxrmate-rrg24/cxrmate-rrg24"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
model = transformers.AutoModel.from_pretrained(model_path, trust_remote_code=True)

In [ ]:
transforms = v2.Compose(
    [
        v2.PILToTensor(),
        v2.Grayscale(num_output_channels=3),
        v2.Resize(size=model.config.encoder.image_size, antialias=True),
        v2.CenterCrop(size=[model.config.encoder.image_size]*2),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=model.config.encoder.image_mean, std=model.config.encoder.image_std),
    ]
)

In [ ]:
# Define a function that will apply the transformation
def apply_transforms(examples):
    # Apply the transformations to each image in the batch
    transformed_images = [transforms(img) for img in examples['image']]
    examples['image'] = transformed_images
    return examples

In [ ]:
# small = hf_dataset.select(range(20)).map(apply_transforms, batched=True, remove_columns=["image_path"])

In [ ]:
# # Custom collate function
# def collate_fn(batch):
#     images = [torch.tensor(item['image']) for item in batch]
#     sids = [item["study_id"] for item in batch]
#     # Stack images into a single tensor
#     images = torch.stack(images)
#     sids = [item["study_id"] for item in batch]
#     # Optionally, handle other fields if needed
#     # You can add code here to handle other fields in the batch if needed
#     return {'image': images, 'sids': sids}

# # Create the DataLoader
# dataloader = DataLoader(small, batch_size=2, collate_fn=collate_fn)

# for data in dataloader:
#     print(data['image'].shape)
#     print(data['sids'])

In [ ]:
hf_dataset = hf_dataset.select(range(100)).map(apply_transforms, batched=True, batch_size=16, num_proc=1, remove_columns="image_path")

In [ ]:
mbatch_size = 16
# Custom collate function
def collate_fn(batch):
    images = [torch.tensor(item['image']) for item in batch]
    sids = [item["study_id"] for item in batch]
    # Stack images into a single tensor
    images = torch.stack(images)
    sids = [item["study_id"] for item in batch]
    # Optionally, handle other fields if needed
    # You can add code here to handle other fields in the batch if needed
    return {'images': images, "sids": sids}
    
dataloader = DataLoader(hf_dataset, batch_size=mbatch_size, collate_fn=collate_fn)
batch = next(iter(dataloader))

In [ ]:
i

In [ ]:
output_ids = model.generate(
    pixel_values=batch['images'].unsqueeze(1),
    max_length=512,
    num_beams=4,
    bad_words_ids=[[tokenizer.convert_tokens_to_ids('[NF]')], [tokenizer.convert_tokens_to_ids('[NI]')]],
)
findings, impression = model.split_and_decode_sections(output_ids, tokenizer)

In [ ]:
results = {
    "study_id" : [],
    "findings" : [],
    "impression" : []
}
for batch in dataloader:
    batch_ids = model.generate(
        pixel_values=batch['images'],
        max_length=512,
        num_beams=4,
        bad_words_ids=[[tokenizer.convert_tokens_to_ids('[NF]')], [tokenizer.convert_tokens_to_ids('[NI]')]],
    )
    batch_findings, batch_impression = model.split_and_decode_sections(batch_ids, tokenizer)
    results["study_id"].extend(batch["study_id"])
    results["findings"].extend(batch_findings)
    results["impression"].extend(batch_impression)


In [ ]:
xrem_path = "/opt/gpudata/anirudh/git-repos/X-REM/"

In [ ]:
pretrain_sids = pd.read_csv(os.path.join(xrem_path, "mimic_pretrain_study_id.csv"))

In [ ]:
pretrain_sids.columns

In [ ]:
challenge_sids = pd.read_csv(os.path.join(xrem_path, "mimic_challenge_study_id.csv"))

In [ ]:
challenge_sids.columns

In [ ]:
len(pretrain_sids["study_id"])

In [ ]:
len(challenge_sids["study_id"])

In [ ]:
inf_sids = inf_only_df["study_id"].tolist()

In [ ]:
len(set(challenge_sids["study_id"].tolist()) - set(inf_sids))

In [ ]:
train_sids = retrieval_df[retrieval_df["split"] == "retrieval"]["study_id"].tolist()

In [ ]:
len(set(challenge_sids["study_id"].tolist()) - set(train_sids))

In [ ]:
itm_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/x-rem/itm_results_temp.csv")
filt_itm_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/x-rem/final_results_filtered.csv")

In [ ]:
itm_df.head()

In [ ]:
filt_itm_df.head()

In [ ]:
print(filt_itm_df.iloc[0]["Report Impression"])

In [ ]:
print(filt_itm_df.iloc[0]["filtered"])

In [ ]:
len(filt_itm_df)

In [ ]:
naive_df=pd.read_csv("/opt/gpudata/rrg-data-2/exp-prompt-true/Mistral-7B-Instruct-v0.3_exact_true-label_naive_top-5_both.csv")

In [ ]:
naive_df.iloc[0]["study_id"]

In [ ]:
print(naive_df.iloc[0]["prompt"])

In [ ]:
simple_df=pd.read_csv("/opt/gpudata/rrg-data-2/exp-prompt-true/Mistral-7B-Instruct-v0.3_exact_true-label_simple_top-5_both.csv")

In [ ]:
simple_df.iloc[0]["study_id"]

In [ ]:
print(simple_df.iloc[0]["prompt"])

In [ ]:
verbose_df=pd.read_csv("/opt/gpudata/rrg-data-2/exp-prompt-true/Mistral-7B-Instruct-v0.3_exact_true-label_verbose_top-5_both.csv")

In [ ]:
verbose_df.iloc[0]["study_id"]

In [ ]:
print(verbose_df.iloc[0]["prompt"])

In [ ]:
instruct_df=pd.read_csv("/opt/gpudata/rrg-data-2/exp-prompt-true/Mistral-7B-Instruct-v0.3_exact_true-label_instruct_top-5_both.csv")

In [ ]:
instruct_df.iloc[0]["study_id"]

In [ ]:
print(instruct_df.iloc[0]["prompt"])

In [ ]:
inf_findings = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inference_findings_data.csv")

In [ ]:
inf_findings

In [ ]:
inf_impression = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inference_impression_data.csv")

In [ ]:
inf_impression

In [ ]:
# both

findings_intersect_impression = pd.merge(inf_findings, inf_impression, how="inner", on = ["study_id"]).dropna()

In [ ]:
findings_intersect_impression

In [ ]:
result_path = "/opt/gpudata/rrg-data-2/inference-all/inf-results"

def create_actual_report(row):
    return f"Findings:\n{row['actual_findings']}\nImpression:\n{row['actual_impression']}"

def create_generated_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"


for foldername in os.listdir(result_path):
    print(f"Process folder : {foldername}")
    folderpath = os.path.join(result_path, foldername)
    if foldername not in ["cxr-llava", "cxr-redone", "pragmatic"]:
        for filename in os.listdir(folderpath):
            if filename == "generations.csv":
                df = pd.read_csv(os.path.join(folderpath, filename))
                # print(df)
                cols = df.columns
                if "findings" in cols:
                    gen_find = inf_findings.merge(df, on="study_id")
                    final_df = gen_find[["study_id", "actual_findings" , "findings"]]
                    output_name = "generations_findings.csv"

                    final_df.rename(columns={
                        "study_id":"study_id", 
                        "actual_findings" : "actual_text", 
                        "findings" : "generated_text"
                        }).to_csv(
                            os.path.join(folderpath, output_name),
                            index=False
                        )
                    
                if "impression" in cols:
                    gen_imp = inf_impression.merge(df, on="study_id")
                    final_df = gen_imp[["study_id", "actual_impression" , "impression"]]
                    output_name = "generations_impression.csv"

                    final_df.rename(
                            columns={
                            "study_id" : "study_id",
                            "actual_impression" : "actual_text",
                            "impression" : "generated_text"
                        }
                    ).to_csv(
                        os.path.join(folderpath, output_name),
                        index=False
                    )

                if "findings" and "impression" in cols:
                    gen_rep = findings_intersect_impression.merge(df, on="study_id")
                    gen_rep["actual_report"] = gen_rep.apply(create_actual_report, axis=1)
                    gen_rep["generated_report"] = gen_rep.apply(create_generated_report, axis=1)
                    final_df = gen_rep[["study_id", "actual_report", "generated_report"]]
                    output_name = "generations_full_reports.csv"

                    final_df.rename(
                        columns={
                            "study_id" : "study_id",
                            "actual_report" : "actual_text",
                            "generated_report" : "generated_text"
                        }
                    ).to_csv(
                        os.path.join(folderpath, output_name),
                        index=False
                    )
